In [1]:
from utils.molecule_tools import get_molecule
from utils.cs_vqe_tools_original import greedy_dfs
import json

/home/ucaptwe/anaconda3/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/ucaptwe/anaconda3/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.optimizers',


In [2]:
file = 'molecule_data'
with open('data/'+file+'.json', 'r') as json_file:
    molecule_data = json.load(json_file)

speciesname_list = list(molecule_data.keys())
len(speciesname_list)

26

In [3]:
speciesname_list[-2:]

['NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET']

In [4]:
model_data = {}
for speciesname in speciesname_list[-2:-1]:
    print(speciesname)
    molecule = get_molecule(speciesname, taper=True)
    num_qubits  = molecule['num_qubits']
    ham         = molecule['hamiltonian']
    uccsd       = molecule['uccsdansatz']
    hf_config   = molecule['hf_config']
    num_tapered = molecule['num_tapered']
    true_gs_nrg = molecule['true_gs_nrg']
    true_gs_vec = molecule['true_gs_vec']
    
    for p in ham.keys():
        ham[p] = ham[p].real    
    for p in uccsd.keys():
        uccsd[p] = uccsd[p].imag
    
    terms_noncon = greedy_dfs(ham, 10, criterion='size')[-1]
    
    model_data[speciesname] = {"ham":ham,
                               "uccsd":uccsd,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg}
                               #"true_gs_vec":true_gs_vec}

NH_STO-3G_SINGLET
*** sector saved, will check tapered ground state energy matches target problem ***
Attempting to taper 12 --> 8 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (-1, -1, 1, 1), tapering successful!



In [6]:
data={}
for k in model_data.keys():
    data[str(k)] = model_data[k]
    
data

{'NH_STO-3G_SINGLET': {'ham': {'IIIIIIII': -35.009718491833496,
   'ZIIIIIII': 0.44108250196018256,
   'IZIIIIII': 0.44108250196018256,
   'ZZIIIIII': 0.16750770481517488,
   'IIZIIIII': -3.469446951953614e-16,
   'ZIZIIIII': -0.007562718988107507,
   'IZZIIIII': 0.007562718988107505,
   'IIIZIIII': -2.983724378680108e-16,
   'ZIIZIIII': -0.007562718988107536,
   'IZIZIIII': 0.0075627189881075415,
   'IIZZIIII': -0.020559259236152858,
   'ZIZZIIII': -5.204170427930421e-18,
   'IZZZIIII': 1.734723475976807e-18,
   'IIIIZIII': 0.7947241424222472,
   'ZIIIZIII': 0.1090209393549455,
   'IZIIZIII': 0.1435504262593516,
   'IIZIZIII': -0.01188498649029429,
   'ZIZIZIII': 3.469446951953614e-18,
   'IIIZZIII': -0.011884986490294324,
   'ZIIZZIII': -1.0408340855860843e-17,
   'IIZZZIII': 1.734723475976807e-18,
   'ZIZZZIII': 0.20698044572350704,
   'IIIIIZII': 0.7947241424222469,
   'ZIIIIZII': 0.1435504262593516,
   'IZIIIZII': 0.1090209393549455,
   'IIZIIZII': 0.011884986490294297,
   'IZZIIZ

In [9]:
model_data

{}

In [27]:
#data={}
#for k in model_data.keys():
#    data[str(k)] = model_data[k]
#     
#with open("data/model_data.json", "w") as outfile: 
#    json.dump(data, outfile)

In [21]:
len(speciesname_list)

24